In [1]:
#import pandas as pd
import polars as pl
from datetime import datetime
import json

import warnings
warnings.filterwarnings('ignore')

fase = '04_crear_train_test'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

#entradas
path_lag = gen_config['path_lag']
path_overall_prod_stats = gen_config['path_overall_prod_stats']
#salidas
path_train = gen_config['path_train']
path_test = gen_config['path_test']
path_futuro = gen_config['path_futuro']

#variables config
periodo_train_start = gen_config['var_periodo_train_start']
periodo_train_end = gen_config['var_periodo_train_end']
periodo_test = gen_config['var_periodo_test']
periodo_futuro = gen_config['var_periodo_futuro']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
----------------------------------------04_crear_train_test-----------------------------------------


In [4]:
overall_prod_stats = pl.read_parquet(f"{folder}/{path_overall_prod_stats}")
overall_prod_stats = overall_prod_stats[['product_id','tot_weight', 'avg_weight', 'med_weight']]
df = pl.read_parquet(f"{folder}/{path_lag}")
df = df.join(overall_prod_stats, on=['product_id'], how='left')

print(f"Tamaño datos con features: {df.shape}")

Tamaño datos con features: (2293481, 189)


In [5]:
df = df.with_columns([
    pl.col('primer_periodo').cast(pl.Int32),
    pl.col('ultimo_periodo').cast(pl.Int32),
    pl.col('periodo_dt').cast(pl.Int32),
    (-pl.col("tn_norm").diff(-2).over(["product_id", "customer_id"])).alias(f'tn_futuro')
])


In [6]:
df_train = df.filter((pl.col('periodo') >= periodo_train_start) & (pl.col('periodo') <= periodo_train_end))
df_test = df.filter(pl.col('periodo') == periodo_test)
df_futuro = df.filter(pl.col('periodo') == periodo_futuro)

In [7]:
print(f"{'Shape Train':.<25}: {df_train.shape} para los periodos desde {periodo_train_start} hasta {periodo_train_end}")
print(f"{'Shape Test':.<25}: {df_test.shape} para el periodo {periodo_test}")
print(f"{'Shape Futuro':.<25}: {df_futuro.shape} para el periodo {periodo_futuro}")

Shape Train..............: (2031615, 190) para los periodos desde 201701 hasta 201908
Shape Test...............: (68823, 190) para el periodo 201910
Shape Futuro.............: (53008, 190) para el periodo 201912


In [8]:
df_train.write_parquet(f'{folder}/{path_train}')
df_test.write_parquet(f'{folder}/{path_test}')
df_futuro.write_parquet(f'{folder}/{path_futuro}')

In [13]:
df_train.glimpse()

Rows: 2031615
Columns: 190
$ periodo                               <i64> 201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710
$ product_id                            <i64> 20001, 20001, 20001, 20001, 20001, 20001, 20001, 20001, 20001, 20001
$ customer_id                           <i64> 10001, 10001, 10001, 10001, 10001, 10001, 10001, 10001, 10001, 10001
$ tn                                    <f64> 99.43861, 198.84365, 92.46537, 13.29728, 101.00563, 128.04792, 101.20711, 43.3393, 289.35024, 222.11389
$ primer_periodo                        <i32> 17167, 17167, 17167, 17167, 17167, 17167, 17167, 17167, 17167, 17167
$ ultimo_periodo                        <i32> 18231, 18231, 18231, 18231, 18231, 18231, 18231, 18231, 18231, 18231
$ values                                <u32> 36, 36, 36, 36, 36, 36, 36, 36, 36, 36
$ total_tn                              <f64> 6347.964899999999, 6347.964899999999, 6347.964899999999, 6347.964899999999, 6347.964899999999, 6347.9648999

In [10]:
df_train['max_1']

max_1
bool
true
true
true
true
true
…
true
true
true


In [21]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

----------------------------------------04_crear_train_test-----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



